# 1、Tools概述

- 工具Tools不仅是"肢体"的延伸，更是为"大脑"(大模型)想象力的"翅膀。借助工具,可以让AI应用的能力真正具备无限的可能

## 1.1、核心功能

- Tools的核心功能:
    - **扩展 LLM 能力**: 让LLM突破纯文本的限制,扩展执行任务的能力(如:调用搜索引擎,操作数据库,运行脚本等)
    - **支持智能决策**: LLM 根据用户问题自动选择最合适的工具去完成任务
    - **工具链调用**: LLM 可以组合多个工具完成复杂任务
    - **模块化设计**: 每个Tool专注一个功能,便于复用和组合(如: 搜索工具 + 计算工具 + 天气查询工具)
- LangChain 拥有大量第三方工具。请访问工具集成查看可用工具列表。
https://python.langchain.com/v0.2/docs/integrations/tools/

## 1.2、基本要素

- Tools本质上是封装了特定功能的可调用模块, 是Agent、Chain 或 LLM可用作和世界互动的接口
- Tool通常包含如下几个要素:
    - 要调用的函数
    - name: 工具的名称(字符串类型),在提供给LLM或Agent的工具集中必须是唯一的
    - description: 工具的功能描述(字符串类型),LLM或Agent将使用此描述作为上下文,以此确定工具是否调用
    - 工具要输入的JSON模式
    - return_direct: 是否应将工具结果直接返回给用户(仅对Agent相关),
        - 默认为False,工具执行结果会返回给Agent,让Agent决定下一步操作
        - 当为True时,在调用给定工具后,Agent将停止并将结果直接返回给用户
- 实操步骤:
    - 步骤1: 将name、description 和 JSON数据作为上下文提供给LLM
    - 步骤2: LLM会根据提示词推断出需要调用哪些工具, 并提供具体的调用参数信息
    - 步骤3: 用户需要根据返回的工具调用信息,自行触发相关工具的回调

# 2、Tools自定义方式

## 2.1、@tool装饰器

- 使用@tool装饰器(自定义工具的最简单方式): 装饰器默认使用 **函数名称作为工具名称**, 但可以通过参数 name_or_callable 来覆盖此设置
- 装饰器将使用函数的 文档字符串 作为 工具的描述 ，因此函数必须提供文档字符串

In [19]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field

@tool
def add_number(a: int, b: int) -> int:  # 工具名为
    """加法运算"""
    return a + b

print(f"name: {add_number.name}")   # name: add_number
print(f"description: {add_number.description}")  # description: 加法运算
print(f"args: {add_number.args}")   # args: {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
print(f"return_direct: {add_number.return_direct}")  # return_direct: False
result = add_number.invoke({"a":10,"b":20})
print(f"add_number result: {result}")

print("=" * 20)

class FieldInfo(BaseModel):
    a :int = Field(description="第1个参数")
    b :int = Field(description="第2个参数")

# name_or_callable可覆盖代替函数名来作为工具名称, description可覆盖原描述来作为工具描述
# args_schema 可用于提供更多信息(例如，few-shot示例)或 验证预期参数
@tool(name_or_callable="add_two_number",description="two number add",args_schema=FieldInfo,return_direct=True)
def add_number(a:int,b:int)->int:
    """两个整数相加"""
    return a + b

print(f"name = {add_number.name}")
print(f"description = {add_number.description}")
print(f"args = {add_number.args}")
print(f"return_direct = {add_number.return_direct}")
res = add_number.invoke({"a":100,"b":20})
print(res)

name: add_number
description: 加法运算
args: {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
return_direct: False
add_number result: 30
name = add_two_number
description = two number add
args = {'a': {'description': '第1个参数', 'title': 'A', 'type': 'integer'}, 'b': {'description': '第2个参数', 'title': 'B', 'type': 'integer'}}
return_direct = True
120


## 2.2、StructuredTool.from_function方法

- StructuredTool.from_function()方法 类似于 @tool 装饰器，但允许更多配置和同步/异步实现的规范,而无需太多额外的代码实现

In [32]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field

class BaseFieldInfo(BaseModel):
    id : int = Field(description="请求id")
    query: str = Field(description="待检索关键词")   # 添加对参数的描述


def web_search(id: int,query: str,) -> str:
    return f"{id} ~ {query} ~ Welcome to LangChain"

web_search_tool = StructuredTool.from_function(
    func=web_search,
    name="web_search_func",
    description="get useful information by website search",
    args_schema=BaseFieldInfo,
    return_direct=True  # 为True,在调用给定工具后,Agent将停止并将结果直接返回给用户
                        # 为False,工具执行结果会返回给Agent,让Agent决定下一步操作
)

print(f"name: {web_search_tool.name}")
print(f"description: {web_search_tool.description}")
print(f"args: {web_search_tool.args}")
print(f"return_direct: {web_search_tool.return_direct}")

web_search_tool.invoke({"id":101,"query": "Hello!"})

name: web_search_func
description: get useful information by website search
args: {'id': {'description': '请求id', 'title': 'Id', 'type': 'integer'}, 'query': {'description': '待检索关键词', 'title': 'Query', 'type': 'string'}}
return_direct: True


'101 ~ Hello! ~ Welcome to LangChain'

# 3、Tools代码实践

## 3.1、扩展LLM的能力

- 示例: 创建本地文件,并写入文本内容

In [13]:
from langchain.tools import tool

@tool
def create_file(filename: str, content: str) -> str:
    """创建一个本地文件，并写入内容"""
    with open(filename, "w", encoding="utf-8") as f:
        f.write(content)
    return f"文件 {filename} 已创建"

# 使用工具：创建并写入本地文件
result = create_file.invoke({"filename": "test.txt", "content": "Hello, LangChain Tools!"})
print(result)

文件 test.txt 已创建


## 3.2、LLM智能决策

- 手动调用工具 VS LLM自动选择工具, 代码示例如下:

In [17]:
from langchain_core.tools import tool

# 第一步：定义工具
# 工具1：数学计算
@tool
def calculator(expression: str) -> str:
    """执行数学计算，例如：calculator("2+3") 返回 "5" """
    try:
        result = eval(expression)
        return str(result)
    except:
        return "计算错误"

# 工具2：获取当前时间
@tool
def get_current_time() -> str:
    """获取当前日期和时间"""
    from datetime import datetime
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# 工具列表
tools = [calculator, get_current_time]

print("已定义的工具：")
for t in tools:
    print(f"  - {t.name} ==> {t.description}")

已定义的工具：
  - calculator ==> 执行数学计算，例如：calculator("2+3") 返回 "5"
  - get_current_time ==> 获取当前日期和时间


In [7]:
# 第二步：直接调用工具（手动方式）
print("【方式1：手动调用工具】")
print("-" * 40)

result1 = calculator.invoke({"expression": "15 * 8 + 20"})
print(f"计算 15 * 8 + 20 = {result1}")

result2 = get_current_time.invoke({})
print(f"当前时间: {result2}")

【方式1：手动调用工具】
----------------------------------------
计算 15 * 8 + 20 = 140
当前时间: 2025-11-07 08:37:25


In [9]:
from langchain_classic.agents import create_openai_tools_agent, AgentExecutor
# 让 LLM 自动选择和使用工具（Tools 的核心价值！）
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os, dotenv

print("\n【方式2：LLM 自动选择工具】")
print("-" * 40)

dotenv.load_dotenv()
# 创建 LLM
llm = ChatOpenAI(
    model = "deepseek-chat",
    base_url = os.environ["BASE_URL"],
    api_key = os.environ["DEEPSEEK_API_KEY"],
    temperature=0
)

# 创建提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你可以使用工具来回答用户。根据问题选择合适的工具。"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
    ])
    
# 创建 agent（将工具绑定到 LLM）
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    
# 示例1：LLM 自动选择计算器工具
print("问题：计算 25 * 4 + 10 等于多少？")
result = agent_executor.invoke({"input": "计算 25 * 4 + 10 等于多少？"})
print(f"答案：{result['output']}\n")
    
# 示例2：LLM 自动选择时间工具
print("问题：现在几点了？")
result = agent_executor.invoke({"input": "现在几点了？"})
print(f"答案：{result['output']}\n")
    




【方式2：LLM 自动选择工具】
----------------------------------------
这就是 Tools 的核心价值：LLM 可以根据问题自动选择工具！

问题：计算 25 * 4 + 10 等于多少？


> Entering new AgentExecutor chain...

Invoking: `calculator` with `{'expression': '25 * 4 + 10'}`
responded: 我来帮你计算 25 * 4 + 10 等于多少。

110计算结果：25 × 4 + 10 = 110

计算过程：
- 25 × 4 = 100
- 100 + 10 = 110

> Finished chain.
答案：计算结果：25 × 4 + 10 = 110

计算过程：
- 25 × 4 = 100
- 100 + 10 = 110

问题：现在几点了？


> Entering new AgentExecutor chain...

Invoking: `get_current_time` with `{}`
responded: 我来帮您查看当前时间。

2025-11-07 08:48:58现在是2025年11月7日，上午8点48分58秒。

> Finished chain.
答案：现在是2025年11月7日，上午8点48分58秒。



- 没有Tools VS 有Tools

**Tools 的核心价值**：

| 问题 | 没有 Tools | 有了 Tools        |
|------|------|-----------------|
|                  | LLM 只能"说"（生成文本） | LLM 可以"做"（执行任务） |
| 25*4等于多少？ | 可能算错  | 自动调用计算器 → 准确结果  |
| 现在几点？| 不知道真实时间 | 自动调用时间工具 → 真实时间 |

**关键点**：LLM 会根据问题**自动选择**合适的工具，无需手动指定！


## 3.3、工具链调用

- 工具链调用: 让 大模型 组合多个工具完成复杂任务

In [34]:
# 让大模型依次调用 数学运算、温度转换、格式化结果 三个工具
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os

# 定义多个工具
@tool
def calculator(expression: str) -> str:
    """执行数学计算，例如：calculator("2+3") 返回 "5" """
    try:
        result = eval(expression)
        return str(result)
    except:
        return "计算错误"

@tool
def temperature_converter(value: float, unit: str) -> str:
    """温度单位转换。unit='c'表示摄氏度转华氏度，unit='f'表示华氏度转摄氏度"""
    if unit.lower() == 'c':
        fahrenheit = value * 9/5 + 32
        return f"{value}°C = {fahrenheit:.2f}°F"
    else:
        celsius = (value - 32) * 5/9
        return f"{value}°F = {celsius:.2f}°C"

@tool
def format_result(value: str, format_type: str) -> str:
    """格式化结果。format_type='bold'加粗，'underline'下划线，'normal'普通"""
    if format_type == 'bold':
        return f"**{value}**"
    elif format_type == 'underline':
        return f"__{value}__"
    else:
        return value

# 工具列表
tools = [calculator, temperature_converter, format_result]
llm = ChatOpenAI(
    model = "deepseek-chat",
    base_url = os.environ["BASE_URL"],
    api_key = os.environ["DEEPSEEK_API_KEY"],
    temperature=0
)
print("已定义的工具：")
for t in tools:
    print(f"  - {t.name} =》 {t.description}")

print("\n" + "=" * 60)
print("【演示：LLM 组合多个工具完成复杂任务】")
print("=" * 60)

# 创建 LLM 和 Agent

prompt = ChatPromptTemplate.from_messages([
    ("system", "你可以使用工具来帮助用户。根据问题选择合适的工具，可以组合多个工具。"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 示例：组合使用多个工具
print("\n问题：先计算 100 * 2，然后把结果当作摄氏度转换成华氏度，最后用加粗格式显示")
print("-" * 60)
result = agent_executor.invoke({
    "input": "先计算 100 * 2，然后把结果当作摄氏度转换成华氏度，最后用加粗格式显示"
})
print(f"\n最终答案：{result['output']}")


已定义的工具：
  - calculator: 执行数学计算，例如：calculator("2+3") 返回 "5"
  - temperature_converter: 温度单位转换。unit='c'表示摄氏度转华氏度，unit='f'表示华氏度转摄氏度
  - format_result: 格式化结果。format_type='bold'加粗，'underline'下划线，'normal'普通

【演示：LLM 组合多个工具完成复杂任务】

问题：先计算 100 * 2，然后把结果当作摄氏度转换成华氏度，最后用加粗格式显示
------------------------------------------------------------


> Entering new AgentExecutor chain...

Invoking: `calculator` with `{'expression': '100 * 2'}`
responded: 我来按照您的要求逐步完成这个任务。

首先计算 100 * 2：

200
Invoking: `temperature_converter` with `{'value': 200, 'unit': 'c'}`
responded: 计算结果是 200。现在将 200 摄氏度转换为华氏度：

200.0°C = 392.00°F
Invoking: `format_result` with `{'value': '200°C = 392.00°F', 'format_type': 'bold'}`
responded: 最后用加粗格式显示结果：

**200°C = 392.00°F**完成！以下是完整的结果：

**200°C = 392.00°F**

> Finished chain.

最终答案：完成！以下是完整的结果：

**200°C = 392.00°F**


## 3.4、如何确定工具的调用情况

In [44]:
from langchain_community.tools import MoveFileTool
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.messages import HumanMessage, AIMessage

# 3.定义工具
tools = [MoveFileTool()]
# 4.这里需要将工具转换为openai函数，后续再将函数传入模型调用
functions = [convert_to_openai_function(t) for t in tools]
# print(functions[0])
# 5. 提供大模型调用的消息列表
messages = [HumanMessage(content="将本目录下的test.txt文件移动到桌面C:\\Users\\xyl\\Desktop")]

# 6.模型使用函数
response = llm.invoke(
    input = messages,    # 大模型会调用工具,返回具体该如何操作的回答
    # [HumanMessage(content="今天的天气怎么样？")],   # 大模型认为无需调用工具(用户问题和工具无关)
    functions=functions
)
print(response)

# print("====================")
#
import json

if "function_call" in response.additional_kwargs:
    tool_name = response.additional_kwargs["function_call"]["name"]
    tool_args = json.loads(messages.additional_kwargs["function_call"]["arguments"])
    print(f"调用工具: {tool_name}, 参数: {tool_args}")
else:
    print("模型回复:", response.content)

content='我可以帮您移动文件。不过，我需要先确认一下当前目录的位置，因为您提到的是"本目录下"。\n\n您可以使用以下命令来移动文件：\n\n**方法1: 使用绝对路径**\n```cmd\nmove test.txt "C:\\Users\\xyl\\Desktop\\"\n```\n\n**方法2: 如果当前用户就是xyl**\n```cmd\nmove test.txt "%USERPROFILE%\\Desktop\\"\n```\n\n**方法3: 使用相对路径（如果当前目录就是test.txt所在目录）**\n```cmd\nmove test.txt "%USERPROFILE%\\Desktop\\"\n```\n\n请先确认：\n1. 您当前在命令行中的位置是否包含test.txt文件\n2. 您的用户名确实是xyl\n\n您可以使用 `dir` 命令查看当前目录下的文件，确认test.txt是否存在，然后执行相应的移动命令。\n\n需要我帮您检查当前目录吗？' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 182, 'prompt_tokens': 20, 'total_tokens': 202, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 20}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '1b8b513f-9c90-46bc-8aa6-be96b75736c2', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--a58bb811-8487-4f91-b8c3

In [ ]:
# - 情况1：大模型决定调用工具
# 如果模型认为需要调用工具（如 MoveFileTool ），返回的 message 会包含：
#     - content : 通常为空（因为模型选择调用工具，而非生成自然语言回复）
#     - additional_kwargs : 包含工具调用的详细信息:

AIMessage(
    content='', # 无自然语言回复
    additional_kwargs={
        'function_call': {
            'name': 'move_file', # 工具名称
            'arguments':
            '{"source_path":"a","destination_path":"/Users/YourUsername/Desktop/a"}' # 工具参数
            }
        }
    )

In [ ]:
# - 情况2：大模型不调用工具
# 如果模型认为无需调用工具（例如用户输入与工具无关），返回的 message 会是普通文本回复

AIMessage(
    content='我没有找到需要移动的文件。', # 自然语言回复
    additional_kwargs={'refusal': None} # 无工具调用
)